<a href="https://colab.research.google.com/github/RemiCailliot/Deep_Learning_Project/blob/main/lstm_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [3]:

# load ascii text and covert to lowercase
filename = "/content/drive/MyDrive/Colab files/001ssb.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

In [4]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [5]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  1607894
Total Vocab:  53


In [6]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  1607794


In [7]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [8]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [18]:
# define the checkpoint
filepath="/content/drive/MyDrive/Colab files/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
filename = "/content/drive/MyDrive/Colab files/weights-improvement-05-1.7963.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [19]:
model.fit(X, y, epochs=5, batch_size=128, callbacks=callbacks_list)

Epoch 1/5
12560/12561 [============================>.] - ETA: 0s - loss: 1.7854
Epoch 1: loss improved from inf to 1.78538, saving model to /content/drive/MyDrive/Colab files/weights-improvement-01-1.7854.hdf5
12561/12561 [==============================] - 444s 35ms/step - loss: 1.7854
Epoch 2/5
12561/12561 [==============================] - ETA: 0s - loss: 1.7736
Epoch 2: loss improved from 1.78538 to 1.77357, saving model to /content/drive/MyDrive/Colab files/weights-improvement-02-1.7736.hdf5
12561/12561 [==============================] - 438s 35ms/step - loss: 1.7736
Epoch 3/5
12561/12561 [==============================] - ETA: 0s - loss: 1.7643
Epoch 3: loss improved from 1.77357 to 1.76432, saving model to /content/drive/MyDrive/Colab files/weights-improvement-03-1.7643.hdf5
12561/12561 [==============================] - 441s 35ms/step - loss: 1.7643
Epoch 4/5
12560/12561 [============================>.] - ETA: 0s - loss: 1.7553
Epoch 4: loss improved from 1.76432 to 1.75528, sav

In [20]:
# load the network weights
filename = "/content/drive/MyDrive/Colab files/weights-improvement-05-1.7526.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [21]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [25]:
# pick a random seed
# Load Larger LSTM network and generate text
import sys
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" w. beyond the wall, the 
shadows lengthen. cotter pyke writes of vast herds of elk, streaming south  "
of the saven shaeow of the 
sooe and the sare and she had so lent the saaeee of the saven shate of the saven sarea wooden haar and 
she sas and she said the sare and she said the sase thi was the sare and she had so lent the sase and 
she said the sase thi was the sare and she had so lent the saaeee of the sase and she had so lent the 
sooe then the sase and she had so lent the saaeee of the sase and the sare and she had so lent the 
sooe then the sase and she had so lent the saaeee of the sase and the sare and she had so lent the 
sooe then the sase and she had so lent the saaeee of the sase and the sare and s

KeyboardInterrupt: ignored